In [1]:
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import cv2
import pandas as pd
import numpy as np
import datetime

from keras.models import Sequential
from PIL import Image    
from tensorflow import keras
from keras import layers
from keras import utils
from glob import glob

In [2]:
!echo 0 | sudo tee -a /sys/bus/pci/devices/0000\:00\:04.0/numa_node

/bin/bash: /home/paul025220/anaconda3/envs/itoi/lib/libtinfo.so.6: no version information available (required by /bin/bash)
0


In [3]:
import tensorflow as tf

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2022-12-07 09:33:06.915703: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 09:33:07.758658: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 14640 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9177844548952321424
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 15352135680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9174408838323690371
 physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
 xla_global_id: 416903419]

In [4]:
# dataset_name = "mnist"
# 반복할 횟수(1이상으로 잡아야 돌아갑니다.)
# dataset_repetitions = 3
dataset_repetitions = 1
# Epoch
num_epochs = 50
# num_epochs = 1

# resize시 정할 이미지
# image_size = 256
image_size = 256
# 데이터셋 컬러여부
# image_colored = 0
image_colored = 1
channel = 3


# KID = Kernel Inception Distance, see related section
kid_image_size = 75
# kid_image_size = 128
kid_diffusion_steps = 5
# kid_diffusion_steps = 10
plot_diffusion_steps = 20
# plot_diffusion_steps = 100
one_plot_diffusion_steps = 20
# one_plot_diffusion_steps = 100

# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
widths = [32, 64, 96, 128]
block_depth = 2

# optimization
batch_size = 1
# batch_size = 16
ema = 0.999
learning_rate = 2e-5
weight_decay = 1e-4

# class
num_class = 2
class_embedding_dims = 32

#dataset = tf.data.Dataset.from_generator(Gen,(tf.float32, tf.int8), ((256,256,6), (8)))
#val_dataset = tf.data.Dataset.from_generator(Gen,(tf.float32, tf.int8), ((256,256,6),(8)))


In [5]:
# http://www.kwangsiklee.com/2018/11/keras%EC%97%90%EC%84%9C-sequence%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%98%EC%97%AC-%EB%8C%80%EC%9A%A9%EB%9F%89-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B-%EC%B2%98%EB%A6%AC%ED%95%98%EA%B8%B0/
# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.

class Gen(tf.keras.utils.Sequence):

    def __init__(self, data_path, labels, batch_size= batch_size, n_channels=channel, n_classes=num_class, shuffle=True, image_size = image_size):
        self.data_path, self.labels = data_path, labels# 데이터셋 경로, 데이터 라벨 리스트
        self.batch_size = batch_size # 배치사이즈 설정
        self.n_channels = n_channels # 채널 수
        self.n_classes = n_classes # 클래스 갯수
        self.shuffle = shuffle # 각 epoch마다 새로운 order를 만들어냄
        self.image_size = image_size
        self.on_epoch_end() # 각 에포크의 맨 처음과 끝에 실행됨. 
        

    def __len__(self):
        return math.ceil(len(self.data_path) / self.batch_size)

    def __getitem__(self, index):
        print('index : ', index)
        indexes =  self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        print(self.indexes)
        print('indexes : ', indexes)
        # list_IDs_temp = [self.labels[k] for k in indexes]
        X, y = self.__data_generation(indexes)
        y = tf.one_hot(indices = y, depth = self.n_classes, dtype = tf.int64)
        y = np.array(y)
        print("X type", type(X))
        print("y type", type(y))
        
        return (X, y)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.data_path))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)  

    def crop_img (self, img, w, h):
        start = ((w // 2 - 128) , (h // 2 - 128))
        end = ((w // 2 + 128), (h // 2 + 128))
        crop_img = img[start[1]:end[1], start[0]:end[0]]
        return np.clip(crop_img/255.0, 0.0, 1.0)
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.image_size, self.image_size, self.n_channels))
        y = np.empty((self.batch_size) , dtype = int) # 메모리만 할당 받고 메모리 초기화는 하지 않는 함수 메모리 초기화를 하려면 zeros를 사용해야함.
        print('self.batch_size :', self.batch_size)
        print('X.shape :', X.shape)
        # Generate data
        for i, ID in enumerate(indexes):
            img = cv2.imread(self.data_path[ID])
            h,w,c = img.shape
            crop = self.crop_img(img, w, h)

            X[i, ] = crop
            y[i] = self.labels[ID]
        return X, y
            

In [6]:
# 확인용 코드
def concat_csv(csv_lists):
    
    for i, csv in enumerate(csv_lists):
        csv_df = pd.read_csv(csv)
        if i == 0:
            df = csv_df
        else:
            df = pd.concat([df, csv_df])
        df = df.sample(frac = 1).reset_index(drop=True)
    return df
            
def df_to_generator(dataframe):
    data_path = list(dataframe['data_path'])
    labels = list(dataframe['label'])

    g = Gen(data_path, labels, batch_size)
    return g



In [7]:
csv_path = 'sample/'
csv_lists = glob(csv_path + '*.csv')

dataframe = concat_csv(csv_lists)
file_count = len(dataframe)
print('file_count :', file_count)
train_df = dataframe[:int(file_count*0.8)]
display(train_df)

val_df = dataframe[int(file_count*0.8):]
display(val_df)

file_count : 32


,label,data_path
0,1,sample/hanok.jpeg
1,2,sample/hanok2.jpeg
2,0,sample/building.jpeg
3,2,sample/hanok2.jpeg
4,0,sample/building.jpeg
5,0,sample/building.jpeg
6,1,sample/hanok.jpeg
7,1,sample/hanok.jpeg
8,1,sample/hanok.jpeg
9,2,sample/hanok2.jpeg


,label,data_path
25,2,sample/hanok2.jpeg
26,1,sample/hanok.jpeg
27,1,sample/hanok.jpeg
28,0,sample/building.jpeg
29,0,sample/building.jpeg
30,0,sample/building.jpeg
31,2,sample/hanok2.jpeg


In [8]:
train_gen = df_to_generator(train_df)
# print(train_gen)
# train_batch =  next(iter(train_gen))
# print(train_batch)

# print('--------------')

val_gen = df_to_generator(val_df)
# print(val_gen)

# val_batch  = next(iter(val_gen))
# print(val_batch)

In [9]:
print(train_gen)
print(len(train_gen))

25


In [10]:
print(next(iter(train_gen))[0].shape)
print("-------")
print(next(iter(train_gen))[1].shape)

index :  0
indexes :  [7]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
-------
index :  0
indexes :  [7]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 2)


2022-12-07 09:33:19.595356: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14640 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0


In [15]:
for t in iter(train_gen):
    print(t[0].shape)

index :  0
indexes :  [7]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
index :  1
indexes :  [9]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
index :  2
indexes :  [11]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
index :  3
indexes :  [5]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
index :  4
indexes :  [14]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
index :  5
indexes :  [12]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <class 'numpy.ndarray'>
y type <class 'numpy.ndarray'>
(1, 256, 256, 3)
index :  6
indexes :  [8]
self.batch_size : 1
X.shape : (1, 256, 256, 3)
X type <clas

In [ ]:
# 이미지의 너비와 높이 중 짧은 길이에 맞춰서 center crop하는 함수
def preprocess_image(data):
    c_vect = tf.one_hot(data["label"], num_class)
    # Center crop한 이미지를 Hyper parameter에서 정해준 image_size에 맞게 resize 해줍니다.
    # resize시 이미지 훼손을 최대한 방지하기 위해 antialias를 True로 해줍니다.
    image = tf.image.resize(image, size=[image_size, image_size], antialias=True)
    
    # 이미지 Array의 값을 float 0~1로 만들어주고 혹시 모를 특이값을 방지하기 위해 clip을 해줍니다.
    return {'image': tf.clip_by_value(image / 255.0, 0.0, 1.0), 'label' : c_vect}

In [ ]:
class KID(keras.metrics.Metric):
    def __init__(self, name, **kwargs):
        super().__init__(name=name, **kwargs)

        # KID 배치마다 계산되며 배치간의 값들로 평균을 냅니다.
        self.kid_tracker = keras.metrics.Mean(name="kid_tracker")

        # a pretrained InceptionV3 is used without its classification layer
        # transform the pixel values to the 0-255 range, then use the same
        # preprocessing as during pretraining
        self.encoder = keras.Sequential(
            [
                keras.Input(shape=(image_size, image_size, 3)),
                layers.Rescaling(255.0),
                layers.Resizing(height=kid_image_size, width=kid_image_size),
                layers.Lambda(keras.applications.inception_v3.preprocess_input),
                keras.applications.InceptionV3(
                    include_top=False,
                    input_shape=(kid_image_size, kid_image_size, 3),
                    weights="imagenet",
                ),
                layers.GlobalAveragePooling2D(),
            ],
            name="inception_encoder",
        )

    def polynomial_kernel(self, features_1, features_2):
        feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
        return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0

    def update_state(self, real_images, generated_images, sample_weight=None):
        real_features = self.encoder(real_images, training=False)
        generated_features = self.encoder(generated_images, training=False)

        # compute polynomial kernels using the two sets of features
        kernel_real = self.polynomial_kernel(real_features, real_features)
        kernel_generated = self.polynomial_kernel(
            generated_features, generated_features
        )
        kernel_cross = self.polynomial_kernel(real_features, generated_features)

        # estimate the squared maximum mean discrepancy using the average kernel values
        batch_size = tf.shape(real_features)[0]
        batch_size_f = tf.cast(batch_size, dtype=tf.float32)
        mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (
            batch_size_f * (batch_size_f - 1.0)
        )
        mean_kernel_generated = tf.reduce_sum(
            kernel_generated * (1.0 - tf.eye(batch_size))
        ) / (batch_size_f * (batch_size_f - 1.0))
        mean_kernel_cross = tf.reduce_mean(kernel_cross)
        kid = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross

        # update the average KID estimate
        self.kid_tracker.update_state(kid)

    def result(self):
        return self.kid_tracker.result()

    def reset_state(self):
        self.kid_tracker.reset_state()


In [ ]:
def sinusoidal_embedding(x):
    embedding_min_frequency = 1.0
    frequencies = tf.exp(
        tf.linspace(
            tf.math.log(embedding_min_frequency),
            tf.math.log(embedding_max_frequency),
            embedding_dims // 2,
        )
    )
    angular_speeds = 2.0 * math.pi * frequencies
    embeddings = tf.concat(
        [tf.sin(angular_speeds * x), tf.cos(angular_speeds * x)], axis=3
    )
    return embeddings

In [ ]:
def ResidualBlock(width):
    def apply(x):
        input_width = x.shape[3]
        if input_width == width:
            residual = x
        else:
            residual = layers.Conv2D(width, kernel_size=1)(x)
        x = layers.BatchNormalization(center=False, scale=False)(x)
        x = layers.Conv2D(
            width, kernel_size=3, padding="same", activation=keras.activations.swish
        )(x)
        x = layers.Conv2D(width, kernel_size=3, padding="same")(x)
        x = layers.Add()([x, residual])
        return x

    return apply

In [ ]:
def DownBlock(width, block_depth):
    def apply(x):
        x, skips = x
        for _ in range(block_depth):
            x = ResidualBlock(width)(x)
            skips.append(x)
        x = layers.AveragePooling2D(pool_size=2)(x)
        return x

    return apply


def UpBlock(width, block_depth):
    def apply(x):
        x, skips = x
        x = layers.UpSampling2D(size=2, interpolation="bilinear")(x)
        for _ in range(block_depth):
            x = layers.Concatenate()([x, skips.pop()])
            x = ResidualBlock(width)(x)
        return x

    return apply

In [ ]:
def get_network(image_size, widths, num_class, block_depth):
    noisy_images = keras.Input(shape=(image_size, image_size, 3))
    noise_variances = keras.Input(shape=(1, 1, 1))
    class_vector = keras.Input(shape = (num_class,))
    
    e = layers.Lambda(sinusoidal_embedding)(noise_variances)
#     print('e :', e)
    e = layers.UpSampling2D(size=image_size, interpolation="nearest")(e)
#     print('e :',e)
    
    i = layers.Dense(class_embedding_dims)(class_vector)
#     print('i :',i)
#     i = layers.Reshape((1,1,class_embedding_dims))(i)
    i = i[:,None,None,:]
#     print('i :',i)
    i = layers.UpSampling2D(size=image_size, interpolation="nearest")(i)
#     print('i :',i)
    
    x = layers.Conv2D(widths[0], kernel_size=1)(noisy_images)
    x = layers.Concatenate()([x, i, e])
#     print('x', x)

    skips = []
    for width in widths[:-1]:
        x = DownBlock(width, block_depth)([x, skips])

    for _ in range(block_depth):
        x = ResidualBlock(widths[-1])(x)

    for width in reversed(widths[:-1]):
        x = UpBlock(width, block_depth)([x, skips])

    x = layers.Conv2D(3, kernel_size=1, kernel_initializer="zeros")(x)
#     x = layers.Conv2D(1, kernel_size=1, kernel_initializer="zeros")(x)
    

    return keras.Model([noisy_images, noise_variances, class_vector], x, name="residual_unet")



In [ ]:
class DiffusionModel(keras.Model):
    def __init__(self, image_size, widths, num_class, block_depth):
        super().__init__()

        self.normalizer = layers.Normalization()
        self.network = get_network(image_size, widths, num_class, block_depth)
        self.ema_network = keras.models.clone_model(self.network)
        
#     def call(self, data, training=False):
        
#         pass
    
    def compile(self, **kwargs):
        super().compile(**kwargs)

        self.noise_loss_tracker = keras.metrics.Mean(name="n_loss")
        self.image_loss_tracker = keras.metrics.Mean(name="i_loss")
        self.kid = KID(name="kid")

    @property
    def metrics(self):
        return [self.noise_loss_tracker, self.image_loss_tracker, self.kid]

    def denormalize(self, images):
        # convert the pixel values back to 0-1 range
        images = self.normalizer.mean + images * self.normalizer.variance**0.5
        return tf.clip_by_value(images, 0.0, 1.0)

    def diffusion_schedule(self, diffusion_times):
        # diffusion times -> angles
        start_angle = tf.acos(max_signal_rate)
        end_angle = tf.acos(min_signal_rate)

        diffusion_angles = start_angle + diffusion_times * (end_angle - start_angle)

        # angles -> signal and noise rates
        signal_rates = tf.cos(diffusion_angles)
        noise_rates = tf.sin(diffusion_angles)
        # note that their squared sum is always: sin^2(x) + cos^2(x) = 1

        return noise_rates, signal_rates

    def denoise(self, noisy_images, noise_rates, signal_rates, class_vector, training):
        # print("denoise start")
        # the exponential moving average weights are used at evaluation
        if training:
            network = self.network
        else:
            network = self.ema_network

        # predict noise component and calculate the image component using it
        # pred_noises = network([noisy_images, noise_rates**2], training=training)
        pred_noises = network([noisy_images, noise_rates**2, class_vector], training=training)
        # print("denoise get noises")
    
        pred_images = (noisy_images - noise_rates * pred_noises) / signal_rates
        # print("denoise get images") 
        
        return pred_noises, pred_images

    def reverse_diffusion(self, initial_noise, diffusion_steps, class_vector):
        # reverse diffusion = sampling
        num_images = initial_noise.shape[0]
        step_size = 1.0 / diffusion_steps

        # important line:
        # at the first sampling step, the "noisy image" is pure noise
        # but its signal rate is assumed to be nonzero (min_signal_rate)
        next_noisy_images = initial_noise
        for step in range(diffusion_steps):
            noisy_images = next_noisy_images

            # separate the current noisy image to its components
            diffusion_times = tf.ones((num_images, 1, 1, 1)) - step * step_size
            noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, class_vector, training=False
            )
            # network used in eval mode

            # remix the predicted components using the next signal and noise rates
            next_diffusion_times = diffusion_times - step_size
            next_noise_rates, next_signal_rates = self.diffusion_schedule(
                next_diffusion_times
            )
            next_noisy_images = (
                next_signal_rates * pred_images + next_noise_rates * pred_noises
            )
            # this new noisy image will be used in the next step

        return pred_images

    def generate(self, num_images, diffusion_steps, class_vector):
        # noise -> images -> denormalized images
        initial_noise = tf.random.normal(shape=(num_images, image_size, image_size, 3))
        generated_images = self.reverse_diffusion(initial_noise, diffusion_steps, class_vector)
        generated_images = self.denormalize(generated_images)
        return generated_images

    # def train_step(self, images):
    def train_step(self, data):
        # normalize images to have standard deviation of 1, like the noises
        # print("train_step start")
        
        #images = data['image']
        #label = data['label']
        print('data :', data)
        image, label = data
        print('images :', image)
        print('label :', label)
        
        print('image.shape :', images.shape)
        
        images = self.normalizer(images, training = True)
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        # sample uniform random diffusion times
        diffusion_times = tf.random.uniform(
            shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        # print("train_step here 1")
        # print("noise_rates :", noise_rates)
        # print("signal_rates :", signal_rates)
        # print("noisy_images :", noisy_images)
        
        with tf.GradientTape() as tape:
            # train the network to separate noisy images to their components
            pred_noises, pred_images = self.denoise(
                noisy_images, noise_rates, signal_rates, label, training=True
            )
            # print('start get loss')
            noise_loss = self.loss(noises, pred_noises)  # used for training
            # print('get noise loss')
            image_loss = self.loss(images, pred_images)  # only used as metric
            # print('get image loss')
            
            
        # print("train_step here 2")
        gradients = tape.gradient(noise_loss, self.network.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.network.trainable_weights))

        self.noise_loss_tracker.update_state(noise_loss)
        self.image_loss_tracker.update_state(image_loss)

        # track the exponential moving averages of weights
        for weight, ema_weight in zip(self.network.weights, self.ema_network.weights):
            ema_weight.assign(ema * ema_weight + (1 - ema) * weight)
    
        # KID is not measured during the training phase for computational efficiency
        return {m.name: m.result() for m in self.metrics[:-1]}

    # def test_step(self, images):
    def test_step(self, data):
#         images = data['image']
#         label = data['label']
        images, label = data
        # normalize images to have standard deviation of 1, like the noises
        images = self.normalizer(images, training=False)
        noises = tf.random.normal(shape=(batch_size, image_size, image_size, 3))

        # sample uniform random diffusion times
        diffusion_times = tf.random.uniform(
            shape=(batch_size, 1, 1, 1), minval=0.0, maxval=1.0
        )
        noise_rates, signal_rates = self.diffusion_schedule(diffusion_times)
        # mix the images with noises accordingly
        noisy_images = signal_rates * images + noise_rates * noises

        # use the network to separate noisy images to their components
        pred_noises, pred_images = self.denoise(
            noisy_images, noise_rates, signal_rates, label, training=False
        )

        noise_loss = self.loss(noises, pred_noises)
        image_loss = self.loss(images, pred_images)

        self.image_loss_tracker.update_state(image_loss)
        self.noise_loss_tracker.update_state(noise_loss)

        # measure KID between real and generated images
        # this is computationally demanding, kid_diffusion_steps has to be small
        images = self.denormalize(images)
        generated_images = self.generate(
            num_images=batch_size, diffusion_steps=kid_diffusion_steps, class_vector = label
        )
        self.kid.update_state(images, generated_images)

        return {m.name: m.result() for m in self.metrics}

    # plot random generated images for visual evaluation of generation quality
    def plot_images(self, epoch=None, logs=None, num_rows=2, num_cols=1):
    # def plot_images(self, epoch=None, logs=None, num_rows=1, num_cols=5):
        
        # indices = [0,1,2,3,4,5,6,7,8,9]
        indices = [0,1]
        sample_vector = tf.one_hot(indices, num_class)
        # sample_vector shape (10,10)
        generated_images = self.generate(
            num_images=num_rows * num_cols,
            diffusion_steps=plot_diffusion_steps,
            class_vector =  sample_vector 
        )
        # generated_images shape : (10,32,32,3)

        plt.figure(figsize=(num_cols * 2.0, num_rows * 2.0))
        for row in range(num_rows):
            for col in range(num_cols):
                index = row * num_cols + col
                plt.subplot(num_rows, num_cols, index + 1)
                plt.imshow(generated_images[index])
                plt.axis("off")
        plt.tight_layout()
        plt.show()
        plt.close()
        
    def plot_one_images(self, input_class = 0, epoch=None, logs=None):
        
        index = [input_class]
        sample_vector = tf.one_hot(index, num_class)
        # sample_vector shape : (1,10)
            
        generated_images = self.generate(
            num_images=1,
            diffusion_steps = one_plot_diffusion_steps,
            class_vector =  sample_vector 
        )
        # generated_images shape : (1,32,32,3)
        
        plt.figure(figsize=(6,6))
        plt.imshow(generated_images[0])
        plt.axis("off")
        plt.tight_layout()
        plt.show()
        plt.close()


In [ ]:
# !pip install tensorflow_addons

In [ ]:
import tensorflow_addons as tfa

# create and compile the model
model = DiffusionModel(image_size, widths, num_class, block_depth)
# below tensorflow 2.9:
# pip install tensorflow_addons
# import tensorflow_addons as tfa
# optimizer=tfa.optimizers.AdamW
model.compile(
    optimizer=tfa.optimizers.AdamW(
#     optimizer=keras.optimizers.experimental.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    ),
    loss=keras.losses.MeanSquaredError(),
)
# pixelwise mean absolute error is used as loss


In [ ]:
# save the best model based on the validation KID metric
checkpoint_path = "checkpoint/diffusion_model"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor="val_kid",
    mode="min",
    save_best_only=True,
)

In [ ]:
# calculate mean and variance of training dataset for normalization
# 이미지와 label로 데이터셋의 형태가 변하였기 때문에 image만 가져와서 adapt 시켜줍니다.
# train_image = train_gen.map(lambda x : x, y)
# model.normalizer.adapt(train_gen)# run training and plot generated images periodically

In [ ]:
model.fit(
    train_gen,
    epochs=num_epochs,
    validation_data=val_gen,
    callbacks=[
        keras.callbacks.LambdaCallback(on_epoch_end = model.plot_images),
        checkpoint_callback,
    ],
)


In [ ]:
# load the best model and generate images
model.load_weights(checkpoint_path)
model.plot_images()